## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Ensuring plots display inline in Jupyter Notebook
%matplotlib inline

In [3]:
# Step 1: Loading the Customer Address Data
cust_address = pd.read_excel('Raw_data.xlsx', sheet_name='CustomerAddress')

In [4]:
# Step 2: Exploring the dataset information
cust_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


The data types of the columns look fine. Let's now assess the data quality and apply a data cleaning process wherever necessary to prepare the dataset for analysis.

## Total Records

In [5]:
# Step 3: Displaying total records and columns
print(f"Total records: {cust_address.shape[0]}")
print(f"Total columns: {cust_address.shape[1]}")

Total records: 3999
Total columns: 6


## Numeric Columns and Non-Numeric Columns

In [6]:

# Step 4: Identifying numeric and non-numeric columns
df_numeric = cust_address.select_dtypes(include=[np.number])
df_non_numeric = cust_address.select_dtypes(exclude=[np.number])
print(f"Numeric columns: {df_numeric.columns.values}")
print(f"Non-numeric columns: {df_non_numeric.columns.values}")

Numeric columns: ['customer_id' 'postcode' 'property_valuation']
Non-numeric columns: ['address' 'state' 'country']


## 1. Missing Values Check

Checking for the presence of missing values in the dataset. If missing values are present for a particular feature, then depending on the situation, the feature may either be dropped (in cases where a significant amount of data is missing) or an appropriate value will be imputed in the feature column.

In [7]:
# Step 5: Checking for missing values
print("Missing values in each column:")
print(cust_address.isnull().sum())


Missing values in each column:
customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64


There are no missing values in the datase

#  2. Inconsistency Check in Data

We will check for inconsistencies or typographical errors in the categorical columns. The columns to be checked are 'address', 'postcode', 'state', and 'country

# 2.1 State

In [8]:
# Step 6: Checking for inconsistencies in 'state' column
print("Unique values in 'state' column before standardization:")
print(cust_address['state'].value_counts())


Unique values in 'state' column before standardization:
state
NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: count, dtype: int64


There are inconsistencies in the 'state' column. For New South Wales and Victoria, both the full names and abbreviations are present. To standardize the state names, 'New South Wales' will be replaced with 'NSW', and 'Victoria' will be replaced with 'VIC'.

In [9]:
# Step 7: Standardizing state names to maintain consistency
state_mapping = {'New South Wales': 'NSW', 'Victoria': 'VIC'}
cust_address['state'] = cust_address['state'].replace(state_mapping)

After applying the function, the state names have been standardized, and there are no inconsistencies in the 'state' column.

In [10]:
# Step 8: Verifying changes to the 'state' column after standardization
print("Unique values in 'state' column after standardization:")
print(cust_address['state'].value_counts())

Unique values in 'state' column after standardization:
state
NSW    2140
VIC    1021
QLD     838
Name: count, dtype: int64


## 2.2 Country

In [12]:
cust_address['country'].value_counts()

country
Australia    3999
Name: count, dtype: int64

There are no inconsistencies in the 'country' column.

## 2.3 Postcode

The 'postcode' column looks perfect. There are no inconsistencies or typographical errors in the data.

In [13]:
cust_address[['address', 'postcode', 'state', 'country']].drop_duplicates()

,address,postcode,state,country
0,060 Morning Avenue,2016,NSW,Australia
1,6 Meadow Vale Court,2153,NSW,Australia
2,0 Holy Cross Court,4211,QLD,Australia
3,17979 Del Mar Point,2448,NSW,Australia
4,9 Oakridge Court,3216,VIC,Australia
...,...,...,...,...
3994,1482 Hauk Trail,3064,VIC,Australia
3995,57042 Village Green Point,4511,QLD,Australia
3996,87 Crescent Oaks Alley,2756,NSW,Australia
3997,8194 Lien Street,4032,QLD,Australia


## 3. Duplication Checks
We need to ensure that there are no duplicate records in the dataset, as duplicates can lead to errors in data analysis due to poor data quality. If there are duplicate rows, we will need to drop those records.

To check for duplicate records, we first need to remove the primary key column from the dataset, and then we can apply the drop_duplicates() function provided by Python.



In [14]:
# Step 9: Dropping duplicates based on address-related columns (excluding customer_id)
cust_address_dedupped = cust_address.drop('customer_id', axis=1).drop_duplicates()
print(f"Records after deduplication: {cust_address_dedupped.shape[0]}")


Number of records after removing customer_id (pk), duplicates : 3999
Number of records in origial dataset : 3999


Since both numbers are the same, there are no duplicate records in the dataset.

## 4. Exporting the Cleaned Customer Demographic Dataset to CSV
The customer address dataset is now clean. Therefore, we can export the data to a CSV file to continue our analysis of customer segments by joining it with other tables.

In [15]:
# Step 10: Exporting the cleaned data to a CSV file
cust_address.to_csv('CustomerAddress_Cleaned.csv', index=False)

## 5. Checking for Master-Detail Record Counts 

We will check the master table (CustomerDemographic_Cleaned), which contains the entire customer data, for the Customer IDs that are being dropped from the customer address dataset.

These Customer IDs belong to customers who have an address but are not part of the demographics dataset yet.

In [16]:
# Step 11: Loading the demographic data and verifying customer IDs against address data
cust_demo_detail = pd.read_csv('CustomerDemographic_Cleaned.csv')

In [17]:
# Step 12: Comparing master-detail record counts
print(f"Total Records in Demographic Table: {cust_demo_detail.shape[0]}")
print(f"Total Records in Address Table: {cust_address.shape[0]}")
print(f"In Demographic Table {cust_address.shape[0] - cust_demo_detail.shape[0]} records are getting dropped")

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,70
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,43
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,70
3,4,Talbot,NaN,Male,33,1961-10-03,Missing,IT,Mass Customer,N,No,7.0,62
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Missing,Affluent Customer,N,Yes,8.0,47


In [19]:
# Step 13: Identifying customer IDs missing in demographic data using an outer join

# Perform an outer join on the 'customer_id' column between the customer address and demographic detail datasets
cust_drop = cust_address.merge(cust_demo_detail, on='customer_id', how='outer', indicator=True)

# Print the customer IDs that are not present in both datasets
print("Customer IDs not present in both datasets:")
print(cust_drop[cust_drop['_merge'] != 'both'])



,customer_id,address,postcode,state,country,property_valuation,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,060 Morning Avenue,2016.0,NSW,Australia,10.0,Laraine,Medendorp,Female,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,70.0
1,2,6 Meadow Vale Court,2153.0,NSW,Australia,10.0,Eli,Bockman,Male,81.0,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,43.0
2,3,NaN,NaN,NaN,NaN,NaN,Arlin,Dearle,Male,61.0,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,70.0
3,4,0 Holy Cross Court,4211.0,QLD,Australia,9.0,Talbot,NaN,Male,33.0,1961-10-03,Missing,IT,Mass Customer,N,No,7.0,62.0
4,5,17979 Del Mar Point,2448.0,NSW,Australia,4.0,Sheila-kathryn,Calton,Female,56.0,1977-05-13,Senior Editor,Missing,Affluent Customer,N,Yes,8.0,47.0
